# 🐘 Introduction au Big Data & Traitement Distribué

---

## 📋 Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Avoir suivi le module `06_intro_relational_databases` |

## 🎯 Objectifs du module

À la fin de ce notebook, tu seras capable de :

- ✅ Comprendre ce qu'est le Big Data et ses caractéristiques (5V)
- ✅ Expliquer pourquoi le traitement distribué est nécessaire
- ✅ Décrire l'architecture Hadoop (HDFS, MapReduce, YARN)
- ✅ Comprendre le modèle MapReduce
- ✅ Expliquer pourquoi Spark a remplacé MapReduce

---

> 💡 **Note** : Ce module est théorique. La pratique viendra au module PySpark !

---

## 🌍 1. C'est quoi le Big Data ?

### 📖 Définition

Le **Big Data** désigne des ensembles de données **tellement volumineux et complexes** qu'ils ne peuvent pas être traités par des outils traditionnels (Excel, bases SQL classiques, un seul serveur).

### 📅 Origine

Le terme a émergé dans les années **2000** avec l'explosion :

- 🌐 D'Internet et des réseaux sociaux
- 📱 Des smartphones
- 📡 Des capteurs IoT
- 💳 Des transactions en ligne

### 📊 Ordres de grandeur

```
1 Ko  (Kilooctet)   = 1 page de texte
1 Mo  (Mégaoctet)   = 1 photo HD
1 Go  (Gigaoctet)   = 1 film HD
1 To  (Téraoctet)   = 1 000 films HD
1 Po  (Pétaoctet)   = 1 000 To = 1 million de Go
1 Eo  (Exaoctet)    = 1 000 Po
1 Zo  (Zettaoctet)  = 1 000 Eo
```

### 🏢 Exemples concrets

| Entreprise | Volume de données |
|------------|-------------------|
| Facebook | ~4 **Po** générés par jour |
| Google | ~20 **Po** traités par jour |
| Netflix | ~60 **Po** de vidéos stockées |
| CERN (LHC) | ~1 **Po** par seconde pendant les expériences |

---

## 📊 2. Les 5V du Big Data

Les caractéristiques du Big Data sont souvent résumées par les **5V** :

```
                    ┌─────────────┐
                    │   VOLUME    │
                    │  (quantité) │
                    └──────┬──────┘
                           │
        ┌──────────────────┼──────────────────┐
        │                  │                  │
        ▼                  ▼                  ▼
┌───────────────┐  ┌───────────────┐  ┌───────────────┐
│   VELOCITY    │  │   VARIETY     │  │   VERACITY    │
│   (vitesse)   │  │  (diversité)  │  │  (fiabilité)  │
└───────────────┘  └───────────────┘  └───────────────┘
                           │
                           ▼
                    ┌─────────────┐
                    │    VALUE    │
                    │  (valeur)   │
                    └─────────────┘
```

---

### 📦 Volume — La quantité massive de données

| Défi | Solution |
|------|----------|
| Impossible de stocker sur un seul disque | Stockage distribué (HDFS, S3) |
| Impossible de charger en RAM | Traitement par partitions |

---

### ⚡ Velocity — La vitesse de génération et traitement

| Type | Exemple | Latence |
|------|---------|--------|
| **Batch** | Rapport mensuel | Heures |
| **Near real-time** | Dashboard | Minutes |
| **Real-time / Streaming** | Détection de fraude | Millisecondes |

---

### 🎨 Variety — La diversité des formats

> 💡 **Rappel** : Tu as déjà vu ça dans le module 06 !

| Type | Format | Exemple |
|------|--------|--------|
| **Structuré** | Tables, colonnes fixes | SQL, CSV |
| **Semi-structuré** | Schéma flexible | JSON, XML (MongoDB, Elasticsearch) |
| **Non-structuré** | Pas de schéma | Images, vidéos, texte libre |

---

### ✅ Veracity — La fiabilité des données

| Problème | Impact |
|----------|--------|
| Données manquantes | Résultats biaisés |
| Doublons | Comptages faux |
| Erreurs de saisie | Mauvaises décisions |
| Données obsolètes | Analyses non pertinentes |

> 💡 C'est là que le **Data Engineer** intervient : nettoyer, valider, transformer !

---

### 💎 Value — La valeur extraite

Les données n'ont de valeur que si on peut en tirer des **insights** :
- 📈 Prédictions (ML)
- 📊 Dashboards
- 🔔 Alertes
- 💰 Optimisation business

---

## ⚙️ 3. Pourquoi le traitement distribué ?

### 🚫 Limites d'une machine unique

Imaginons que tu dois traiter **10 To** de logs :

| Ressource | Limite typique | Problème |
|-----------|---------------|----------|
| **RAM** | 64-256 Go | 10 To ne tient pas en mémoire |
| **CPU** | 8-64 cœurs | Traitement séquentiel = trop lent |
| **Disque** | 500 Mo/s lecture | 10 To = 5+ heures juste pour lire |
| **Réseau** | Goulot d'étranglement | Transférer 10 To = des heures |

---

### 📈 Scale-Up vs Scale-Out

```
SCALE-UP (vertical)              SCALE-OUT (horizontal)
─────────────────────            ─────────────────────────

   ┌─────────────┐               ┌───┐ ┌───┐ ┌───┐ ┌───┐
   │             │               │   │ │   │ │   │ │   │
   │   MEGA      │               │ S │ │ S │ │ S │ │ S │
   │  SERVEUR    │      vs       │ 1 │ │ 2 │ │ 3 │ │ 4 │
   │   💪💪💪    │               │   │ │   │ │   │ │   │
   │             │               └───┘ └───┘ └───┘ └───┘
   └─────────────┘               
                                 Cluster de serveurs
   + Plus de RAM                 + Moins cher (commodity)
   + Plus de CPU                 + Scalabilité infinie
   - Très cher $$$               + Tolérance aux pannes
   - Limite physique             - Plus complexe
```

### ✅ Le Big Data utilise le Scale-Out !

Au lieu d'une machine surpuissante, on utilise un **cluster** de machines ordinaires.

---

## 🧠 4. Concepts clés du traitement distribué

### 🔄 Parallélisme vs Distribution

| Concept | Description | Exemple |
|---------|-------------|--------|
| **Parallélisme** | Plusieurs tâches en même temps sur **une machine** | Multi-threading |
| **Distribution** | Tâches réparties sur **plusieurs machines** | Cluster Hadoop/Spark |

---

### 📍 Data Locality — "Amener le code aux données"

```
❌ MAUVAIS : Déplacer les données vers le code
┌────────────────┐                ┌────────────────┐
│   Serveur 1    │  ══10 To══►    │   Serveur 2    │
│   (données)    │   réseau       │    (calcul)    │
└────────────────┘   lent !       └────────────────┘

✅ BON : Déplacer le code vers les données
┌────────────────┐                ┌────────────────┐
│   Serveur 1    │  ◄══code══     │   Serveur 2    │
│ données+calcul │   (petit)      │    (master)    │
└────────────────┘                └────────────────┘
```

> 💡 C'est le principe fondamental de Hadoop et Spark !

---

### 🛡️ Fault Tolerance — Tolérance aux pannes

Dans un cluster de 1000 machines, **des pannes arrivent tous les jours** !

| Stratégie | Description |
|-----------|-------------|
| **Réplication** | Copier les données sur plusieurs nœuds (HDFS : 3 copies) |
| **Checkpointing** | Sauvegarder l'état intermédiaire |
| **Lineage** | Recalculer les données perdues (Spark RDD) |

---

## 🐘 5. L'écosystème Hadoop

**Hadoop** est un framework open-source créé par Yahoo (2006), inspiré des papiers de Google (GFS, MapReduce).

### 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    ÉCOSYSTÈME HADOOP                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────┐       │
│   │  Hive   │  │   Pig   │  │  HBase  │  │  Sqoop  │  ...  │
│   │  (SQL)  │  │(scripts)│  │ (NoSQL) │  │ (import)│       │
│   └────┬────┘  └────┬────┘  └────┬────┘  └────┬────┘       │
│        │            │            │            │             │
│        └────────────┴─────┬──────┴────────────┘             │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │  MapReduce  │  ◄── Traitement         │
│                    │  (calcul)   │                         │
│                    └──────┬──────┘                         │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │    YARN     │  ◄── Ressources         │
│                    │ (scheduler) │                         │
│                    └──────┬──────┘                         │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │    HDFS     │  ◄── Stockage           │
│                    │  (fichiers) │                         │
│                    └─────────────┘                         │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

### 📁 HDFS — Hadoop Distributed File System

Système de fichiers distribué qui stocke les données sur plusieurs machines.

```
Fichier original : data.csv (300 Mo)
                      │
        ┌─────────────┼─────────────┐
        ▼             ▼             ▼
   ┌─────────┐   ┌─────────┐   ┌─────────┐
   │ Block 1 │   │ Block 2 │   │ Block 3 │   (128 Mo chacun)
   │ 128 Mo  │   │ 128 Mo  │   │  44 Mo  │
   └────┬────┘   └────┬────┘   └────┬────┘
        │             │             │
   Répliqué 3x   Répliqué 3x   Répliqué 3x
```

| Caractéristique | Valeur par défaut |
|-----------------|-------------------|
| Taille de bloc | 128 Mo |
| Facteur de réplication | 3 |
| Type d'accès | Write once, read many |

---

### 🎛️ YARN — Yet Another Resource Negotiator

Gestionnaire de ressources du cluster :

- Alloue CPU/RAM aux applications
- Gère la file d'attente des jobs
- Surveille l'état des nœuds

---

## 🔄 6. MapReduce — Le modèle de traitement

**MapReduce** est un modèle de programmation pour traiter de grandes quantités de données en parallèle.

### 📐 Les 3 étapes

| Étape | Action | Parallélisme |
|-------|--------|-------------|
| **Map** | Transformer chaque élément | ✅ Parallèle |
| **Shuffle** | Regrouper par clé | ⚠️ Réseau |
| **Reduce** | Agréger les valeurs | ✅ Parallèle |

---

### 📝 Exemple : Word Count

Compter les occurrences de chaque mot dans un texte.

```
ENTRÉE : "hello world hello"

═══════════════════════════════════════════════════════════════════
                              MAP
═══════════════════════════════════════════════════════════════════

  "hello world hello"   
         │
         ▼
  ┌──────────────────────────────────────┐
  │  (hello, 1)  (world, 1)  (hello, 1)  │
  └──────────────────────────────────────┘

═══════════════════════════════════════════════════════════════════
                            SHUFFLE
═══════════════════════════════════════════════════════════════════

  Regrouper par clé (mot) :
  
  hello → [1, 1]
  world → [1]

═══════════════════════════════════════════════════════════════════
                            REDUCE
═══════════════════════════════════════════════════════════════════

  hello → sum([1, 1]) → 2
  world → sum([1])    → 1

═══════════════════════════════════════════════════════════════════

SORTIE : { "hello": 2, "world": 1 }
```

---

### 🖥️ MapReduce sur un cluster

```
                         ┌─────────────┐
                         │   MASTER    │
                         │  (Driver)   │
                         └──────┬──────┘
                                │
           ┌────────────────────┼────────────────────┐
           │                    │                    │
           ▼                    ▼                    ▼
    ┌─────────────┐      ┌─────────────┐      ┌─────────────┐
    │   NODE 1    │      │   NODE 2    │      │   NODE 3    │
    ├─────────────┤      ├─────────────┤      ├─────────────┤
    │ Données:    │      │ Données:    │      │ Données:    │
    │ "hello"     │      │ "world"     │      │ "hello"     │
    ├─────────────┤      ├─────────────┤      ├─────────────┤
    │ MAP:        │      │ MAP:        │      │ MAP:        │
    │ (hello, 1)  │      │ (world, 1)  │      │ (hello, 1)  │
    └──────┬──────┘      └──────┬──────┘      └──────┬──────┘
           │                    │                    │
           └────────────────────┼────────────────────┘
                                │
                         SHUFFLE (réseau)
                                │
           ┌────────────────────┴────────────────────┐
           ▼                                         ▼
    ┌─────────────┐                           ┌─────────────┐
    │  REDUCER 1  │                           │  REDUCER 2  │
    ├─────────────┤                           ├─────────────┤
    │ hello: [1,1]│                           │ world: [1]  │
    │ → hello: 2  │                           │ → world: 1  │
    └─────────────┘                           └─────────────┘
```

> 💡 Chaque nœud traite **ses données localement** (data locality) !

---

## ⚠️ 7. Limites de MapReduce

### 😓 Problèmes de MapReduce

| Problème | Cause | Impact |
|----------|-------|--------|
| **Lent** | Écrit sur disque entre chaque étape | I/O intensif |
| **Verbose** | Code Java complexe | Productivité basse |
| **Batch only** | Pas de streaming | Pas de temps réel |
| **Pas de cache** | Relit les données à chaque job | Itérations lentes (ML) |

### 💾 Le problème du disque

```
MapReduce : DISQUE → Map → DISQUE → Shuffle → DISQUE → Reduce → DISQUE
                 ↑           ↑              ↑              ↑
                 └───────────┴──────────────┴──────────────┘
                              LENT ! (I/O disque)

Spark :     DISQUE → Map → MÉMOIRE → Shuffle → MÉMOIRE → Reduce
                            ↑                    ↑
                            └────────────────────┘
                              RAPIDE ! (in-memory)
```

---

### 🔥 Exemple : Algorithme itératif (ML)

Pour un algorithme qui fait 10 itérations sur les mêmes données :

| Framework | Comportement | Temps |
|-----------|-------------|-------|
| **MapReduce** | Relit les données du disque 10 fois | 😓 |
| **Spark** | Garde les données en mémoire, itère 10 fois | ⚡ |

---

## ⚡ 8. Spark — L'évolution

**Apache Spark** (2014) a été créé pour résoudre les limitations de MapReduce.

### 📊 Comparaison Hadoop MapReduce vs Spark

| Critère | Hadoop MapReduce | Apache Spark |
|---------|------------------|---------------|
| **Vitesse** | Lent (disque) | 100x plus rapide (mémoire) |
| **Facilité** | Java verbeux | Python, Scala, SQL |
| **Traitement** | Batch only | Batch + Streaming |
| **Itérations** | Lent (relit le disque) | Rapide (cache en RAM) |
| **Écosystème** | Hive, Pig, etc. | Spark SQL, MLlib, GraphX |
| **Stockage** | HDFS | HDFS, S3, Cassandra, etc. |

---

### 🏗️ Architecture Spark

```
┌─────────────────────────────────────────────────────────────┐
│                      APACHE SPARK                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ┌───────────┐ ┌───────────┐ ┌───────────┐ ┌───────────┐  │
│   │ Spark SQL │ │ Streaming │ │   MLlib   │ │  GraphX   │  │
│   │  (SQL)    │ │ (temps    │ │   (ML)    │ │ (graphes) │  │
│   │           │ │   réel)   │ │           │ │           │  │
│   └─────┬─────┘ └─────┬─────┘ └─────┬─────┘ └─────┬─────┘  │
│         │             │             │             │        │
│         └─────────────┴──────┬──────┴─────────────┘        │
│                              │                              │
│                       ┌──────┴──────┐                      │
│                       │ Spark Core  │                      │
│                       │   (RDD)     │                      │
│                       └──────┬──────┘                      │
│                              │                              │
│         ┌────────────────────┼────────────────────┐        │
│         ▼                    ▼                    ▼        │
│   ┌──────────┐        ┌──────────┐        ┌──────────┐    │
│   │   YARN   │        │   Mesos  │        │Standalone│    │
│   └──────────┘        └──────────┘        └──────────┘    │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

> 💡 Spark peut fonctionner **sur YARN** (cluster Hadoop existant) ou en mode standalone.

---

### 🔮 Concepts Spark à venir 

| Concept | Description |
|---------|-------------|
| **RDD** | Resilient Distributed Dataset — collection distribuée |
| **DataFrame** | Comme un tableau avec colonnes (similaire à Pandas) |
| **Transformation** | Opération lazy (map, filter, groupBy) |
| **Action** | Déclenche le calcul (collect, count, show) |
| **Lazy Evaluation** | Rien ne s'exécute tant qu'une action n'est pas appelée |
| **Partition** | Morceau de données traité par un worker |

---

### 🎯 Ce que tu vas apprendre avec PySpark

- Créer et manipuler des DataFrames distribués
- Écrire des transformations SQL-like
- Lire/écrire des fichiers (CSV, Parquet, JSON)
- Optimiser les performances
- Construire des pipelines de données

---

## 📋 Résumé

### Les 5V du Big Data

| V | Défi |
|---|------|
| **Volume** | Stocker et traiter des To/Po |
| **Velocity** | Traiter en temps réel |
| **Variety** | Gérer tous les formats |
| **Veracity** | Assurer la qualité |
| **Value** | Extraire des insights |

### Traitement distribué

| Concept | Retenir |
|---------|--------|
| **Scale-out** | Cluster de machines ordinaires |
| **Data locality** | Amener le code aux données |
| **Fault tolerance** | Réplication, recalcul |

### Hadoop vs Spark

| | Hadoop MR | Spark |
|---|-----------|-------|
| Stockage intermédiaire | Disque | Mémoire |
| Vitesse | Lent | 100x plus rapide |
| Langages | Java | Python, Scala, SQL |

---

## 🧠 Quiz

---

### ❓ Q1. Quels sont les 3V originaux du Big Data ?
a) Vitesse, Valeur, Vérité  
b) Volume, Velocity, Variety  
c) Volume, Validation, Visualisation  
d) Vélocité, Véracité, Valorisation

<details><summary>💡 Réponse</summary>

✅ **b** — Les 3V originaux (Doug Laney, 2001) sont Volume, Velocity, Variety.
</details>

---

### ❓ Q2. Que signifie "Scale-out" ?
a) Augmenter la RAM d'un serveur  
b) Ajouter des machines au cluster  
c) Compresser les données  
d) Réduire la taille du cluster

<details><summary>💡 Réponse</summary>

✅ **b** — Scale-out (horizontal) = ajouter des machines. Scale-up (vertical) = augmenter les ressources d'une machine.
</details>

---

### ❓ Q3. Quel est le principe de "Data Locality" ?
a) Stocker les données localement sur son PC  
b) Déplacer les données vers le serveur de calcul  
c) Déplacer le code vers les données  
d) Compresser les données pour les transférer

<details><summary>💡 Réponse</summary>

✅ **c** — On envoie le code (petit) vers les données (grosses), pas l'inverse.
</details>

---

### ❓ Q4. Quels sont les 3 composants principaux de Hadoop ?
a) HDFS, Spark, Kafka  
b) HDFS, MapReduce, YARN  
c) Hive, Pig, HBase  
d) Map, Shuffle, Reduce

<details><summary>💡 Réponse</summary>

✅ **b** — HDFS (stockage), MapReduce (calcul), YARN (ressources).
</details>

---

### ❓ Q5. Pourquoi Spark est plus rapide que MapReduce ?
a) Il utilise un meilleur algorithme  
b) Il stocke les données intermédiaires en mémoire  
c) Il compresse les données  
d) Il utilise plus de serveurs

<details><summary>💡 Réponse</summary>

✅ **b** — Spark garde les données en mémoire (RAM) au lieu d'écrire sur disque entre chaque étape.
</details>

---

### ❓ Q6. Dans MapReduce, que fait l'étape "Shuffle" ?
a) Trier les données  
b) Regrouper les données par clé  
c) Supprimer les doublons  
d) Compresser les résultats

<details><summary>💡 Réponse</summary>

✅ **b** — Le Shuffle regroupe toutes les valeurs ayant la même clé pour les envoyer au même Reducer.
</details>

---

### ❓ Q7. Combien de copies HDFS fait-il par défaut ?
a) 1  
b) 2  
c) 3  
d) 5

<details><summary>💡 Réponse</summary>

✅ **c** — HDFS réplique chaque bloc 3 fois par défaut pour la tolérance aux pannes.
</details>

---

## 📚 Ressources

- [Hadoop Documentation](https://hadoop.apache.org/docs/stable/)
- [Spark Documentation](https://spark.apache.org/docs/latest/)
- [The Google File System (paper)](https://research.google/pubs/pub51/) — L'article original
- [MapReduce: Simplified Data Processing (paper)](https://research.google/pubs/pub62/) — L'article original

---

## ➡️ Prochaine étape

Tu as maintenant les bases théoriques du Big Data et du traitement distribué. Place à la pratique !

👉 **Module suivant : `09_mongodb_for_data_engineers.ipynb`** — PySpark pour le traitement distribué

---

🎉 **Félicitations !** Tu comprends maintenant pourquoi et comment le Big Data fonctionne.